<a href="https://colab.research.google.com/github/lzhkaya98/SSMGlobal/blob/main/Globalretrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
###模型训练###
#!pip install xlrd
import torch
import torch.nn as nn
import torch.nn.functional as tf
from torch.utils import data
import numpy as np
import xlrd
import random
from sklearn import preprocessing
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import pylab
from sklearn.preprocessing import MinMaxScaler

from google.colab import drive
drive.mount('/content/drive')


#数据集加载
def load_data(StartPo,EndPo,TestProportion,FeatureNum,Shuffle,FilePath):         #样本起始行数，结束行数，测试集占总样本集比重,特征数，是否打乱样本集     #如果Testproportion为0或1就训练集=测试集
    #打开excel文件
    workbook = xlrd.open_workbook(str(FilePath))       #excel路径
    sheet = workbook.sheet_by_name('sheet1')             #sheet表
    Sample = []#总样本集
    train = []#训练集
    test = []#测试集
    TestSetSphere = (EndPo-StartPo+1)*TestProportion  #测试集数目
    TestSetSphere = int(TestSetSphere)#测试集数目
    #获取全部样本集并打乱顺序
    for loadi in range(StartPo-1,EndPo):
        RowSample = sheet.row_values(loadi)
        Sample.append(RowSample)
    if Shuffle == 1:  #是否打乱样本集
        random.shuffle(Sample)  #如果shuffle=1，打乱样本集
    #如果Testproportion为0就训练集=测试集
    if TestProportion == 0 or TestProportion == 1:
        TrainSet = np.array(Sample)          #变换为array
        TestSet = np.array(Sample)
        TrainSet = torch.Tensor(TrainSet)
        TestSet = torch.Tensor(TestSet)
    else:
        #设置训练集
        for loadtraina in Sample[:(EndPo-TestSetSphere)]:
            GetTrainValue = loadtraina
            train.append(GetTrainValue)
        #设置测试集
        for loadtesta in range(-TestSetSphere-1,-1):
            GetTestValue = Sample[loadtesta]
            test.append(GetTestValue)
        #变换样本集
        TrainSet = np.array(train)                  #变换为array
        TestSet = np.array(test)
        TrainSet = torch.Tensor(TrainSet)
        TestSet = torch.Tensor(TestSet)


   #分割特征与目标变量
    x1 , y1 = TrainSet[:,:FeatureNum] , TrainSet[:,-1]
    Tb_train = x1[:,0] ; LST_train = x1[:,1] ; dyn_train = x1[:,2:5] ; sta_train = x1[:,2:5] ; dic_train = x1[:,10:FeatureNum-1]
    x2 , y2 = TestSet[:,:FeatureNum] , TestSet[:,-1]
    Tb_test = x2[:,0] ; LST_test = x2[:,1] ; dyn_test = x2[:,2:5] ; sta_test = x2[:,2:5] ; dic_test = x2[:,10:FeatureNum-1]
    return sta_train , dyn_train, torch.unsqueeze(Tb_train,dim=1), torch.unsqueeze(LST_train,dim=1), dic_train, torch.unsqueeze(y1,dim=1) , sta_test, dyn_test, torch.unsqueeze(Tb_test,dim=1), torch.unsqueeze(LST_test,dim=1), dic_test, torch.unsqueeze(y2,dim=1)

sta_train , dyn_train, Tb_train, LST_train, dic_train, y1 , sta_test, dyn_test, Tb_test, LST_test, dic_test, y2 = load_data(2,65536,0.2,14,1,str('/content/drive/My Drive/train_global.xls'))
#网络超参数设置
#具有两个隐藏层的FNN
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.bn = nn.BatchNorm1d(input_size)
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size,hidden_size)
        self.fc3 = nn.Linear(hidden_size,hidden_size)
        self.fc4 = nn.Linear(hidden_size,hidden_size)
        self.fc5 = nn.Linear(hidden_size,output_size)

    def forward(self,x):
        x = self.bn(x)
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        x = torch.tanh(self.fc4(x))
        x = self.fc5(x)
        return x

def load_array(data_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset,batch_size,shuffle=True)


#微分WCM模型
#动态参数VC，静态参数C
def SCA(w, tau, Tb, LST, dic):
    gamma = torch.exp(-tau / 0.766)    #cos40°=0.766
    sigma_veg = Tb - LST * ((1-w)*(1-gamma)+gamma)
    sigma_soil = LST * gamma * ((1-w)*(1-gamma)-1)
    r = sigma_veg / sigma_soil
    input = torch.cat([r, dic], dim=1)
    return input

#散点图
def scatter_plot_frequency(x, y):
  xy = np.vstack([x,y])
  z = gaussian_kde(xy)(xy)
  # Sort the points by density, so that the densest points are plotted last
  idx = z.argsort()
  x, y, z = x[idx], y[idx], z[idx]
  fig, ax = plt.subplots()
  # 设置参考的1：1虚线参数
  xxx = [0, 0.6]  #[0, 0.6] 40
  yyy = [0, 0.6]
  plt.plot(xxx, yyy, c='0', linewidth=1, linestyle=':', marker='.', alpha=0.3)  # 绘制虚线
  plt.scatter(x, y, c=z, s=20, cmap='Spectral_r')
  z=np.polyfit(x,y,1)
  p=np.poly1d(z)
  pylab.plot(x,p(x),"r")
  plt.colorbar()
  plt.show()

#模型训练
batch_size = 128 #16,32,64...
num_epochs = 3000  #1000,2000...
data_iter = load_array((sta_train , dyn_train, Tb_train, LST_train, dic_train, y1), batch_size)

model_w = Net(3,5,1)   #5 10 1
model_tau = Net(3,5,1)   #8 10 1
model_SM = Net(4,5,1)   #10 10 1
#定义损失函数和优化器
criterion = nn.L1Loss()
trainer1 = torch.optim.SGD(model_SM.parameters(),lr=0.01)
trainer2 = torch.optim.SGD(model_tau.parameters(),lr=0.01)
trainer3 = torch.optim.SGD(model_w.parameters(),lr=0.01)

for epoch in range(num_epochs):
    for sta,dyn,Tb,LST,dic,y in data_iter:
        tau = model_tau(dyn)
        w = model_w(sta) #sta
        input = SCA(w, tau, Tb, LST, dic)
        SM = model_SM(input)

        loss = criterion(SM, y)
#        print(loss)
        model_SM.zero_grad()
        model_tau.zero_grad()
        model_w.zero_grad()
        loss.backward()
        trainer1.step()
        trainer2.step()
        trainer3.step()
#    print(f'epoch {epoch+1}')

#模型保存
torch.save(model_w.state_dict(), str('/content/drive/My Drive/model_w_global4.pth'))
torch.save(model_tau.state_dict(), str('/content/drive/My Drive/model_tau_global4.pth'))
torch.save(model_SM.state_dict(), str('/content/drive/My Drive/model_SM_global4.pth'))


#模型测试
tau_test = model_tau(dyn_test)
w_test = model_w(sta_test)  #dyn_test
input_test = SCA(w_test, tau_test, Tb_test, LST_test, dic_test)
SM_test = model_SM(input_test)
loss = criterion(SM_test, y2)
print(loss)

#模型输出评估
#"""
from sklearn.metrics import r2_score
from scipy.stats import pearsonr

Y_test = y2.numpy()
Y_pred = SM_test.detach().numpy()
#print(Y_test)
#print(Y_pred)
r2 = r2_score(Y_test, Y_pred)
ResidualSquare = (Y_pred - Y_test) ** 2
MSE = np.mean(ResidualSquare)
bias = np.mean(Y_pred - Y_test)
RMSE = np.sqrt(MSE)
ubRMSE = np.sqrt(RMSE ** 2 - bias ** 2)
print(f'R2={r2}')
print(f'bias={bias}')
print(f'RMSE={RMSE}')
print(f'ubRMSE={ubRMSE}')
print("R P", pearsonr(Y_pred[:,0], Y_test[:,0]))
scatter_plot_frequency(Y_test[:,0],Y_pred[:,0])  #生成散点图
#"""

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as tf
from torch.utils import data
import pandas as pd
import numpy as np
import datetime
from datetime import date
import time
import ee
ee.Authenticate()
ee.Initialize(project='ee-whulzh98')
from time import time

from google.colab import drive
drive.mount('/content/drive')

###########################################################模型选择##############################################################################
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.bn = nn.BatchNorm1d(input_size)
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size,hidden_size)
        self.fc3 = nn.Linear(hidden_size,hidden_size)
        self.fc4 = nn.Linear(hidden_size,hidden_size)
        self.fc5 = nn.Linear(hidden_size,output_size)

    def forward(self,x):
        x = self.bn(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)
        return x
#SCA-V模型
def SCA(w, tau, Tb, LST, dic):
    gamma = torch.exp(-tau / 0.766)    #cos40°=0.766
    sigma_veg = Tb - LST * ((1-w)*(1-gamma)+gamma)
    sigma_soil = LST * gamma * ((1-w)*(1-gamma)-1)
    r = sigma_veg / sigma_soil
    input = torch.cat([r, dic], dim=1)
    return input
#模型默认为5,8,10，relu
model_w = Net(5,10,1)   #5 10 1
model_tau = Net(8,10,1)   #8 10 1
model_SM = Net(10,10,1)   #10 10 1

###########################################################数据获取##############################################################################
timerange = ee.Filter.date('2018-01-1', '2018-12-31')
def auxiliarydate_load(lonn,latt,scaled):
  pt= ee.Geometry.Point([lonn,latt])

  dataset1 = ee.Image('COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019')
  barecover = dataset1.select('bare-coverfraction')
  ft1 = barecover.reduceRegions(pt, ee.Reducer.first(),scaled)
  grasscover = dataset1.select('grass-coverfraction')
  ft2 = grasscover.reduceRegions(pt, ee.Reducer.first(),scaled)
  cropscover = dataset1.select('crops-coverfraction')
  ft3 = cropscover.reduceRegions(pt, ee.Reducer.first(),scaled)
  shrubcover = dataset1.select('shrub-coverfraction')
  ft4 = shrubcover.reduceRegions(pt, ee.Reducer.first(),scaled)
  treecover = dataset1.select('tree-coverfraction')
  ft5 = treecover.reduceRegions(pt, ee.Reducer.first(),scaled)
  Bare = ft1.getInfo().get('features')[0]['properties']['first']/100
  Grass = ft2.getInfo().get('features')[0]['properties']['first']/100
  Crops = ft3.getInfo().get('features')[0]['properties']['first']/100
  Shrub = ft4.getInfo().get('features')[0]['properties']['first']/100
  Tree = ft5.getInfo().get('features')[0]['properties']['first']/100

  dataset2 = ee.Image("OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02").select('b0')
  dataset3 = ee.Image("OpenLandMap/SOL/SOL_SAND-WFRACTION_USDA-3A1A1A_M/v02").select('b0')
  dataset4 = ee.Image("OpenLandMap/SOL/SOL_BULKDENS-FINEEARTH_USDA-4A1H_M/v02").select('b0')

  ft6=dataset2.reduceRegions(pt, ee.Reducer.first(),scaled)
  ft7=dataset3.reduceRegions(pt, ee.Reducer.first(),scaled)
  ft8=dataset4.reduceRegions(pt, ee.Reducer.first(),scaled)

  Clay = ft6.getInfo().get('features')[0]['properties']['first']/100
  Sand = ft7.getInfo().get('features')[0]['properties']['first']/100
  Bulk = ft8.getInfo().get('features')[0]['properties']['first']/100

  dataset5 = ee.Image('USGS/SRTMGL1_003').select('elevation')
  slope1 = ee.Terrain.slope(dataset5)
  ft9 = slope1.reduceRegions(pt, ee.Reducer.first(),scaled)
  Slope = ft9.getInfo().get('features')[0]['properties']['first']

  return Bare,Grass,Crops,Shrub,Tree,Clay,Sand,Bulk,Slope

def modisload(lonn,latt,year,month,day,scaled):

  def dataload(dataset):
    M = dataset.filter(timerange)
    pt= ee.Geometry.Point([lonn,latt])
    begin = datetime.date(year,month,day) #开始时间
    d = begin
    delta = datetime.timedelta(days=3)
    oneDate=d.strftime("%Y-%m-%d")
    dd=d+delta
    oneDate_next=dd.strftime("%Y-%m-%d")
    M_Day=M.filterDate(oneDate,oneDate_next).filterBounds(pt)
    ft1=M_Day.first().reduceRegions(pt, ee.Reducer.first(),scaled)
    data1=ft1.getInfo().get('features')[0]['properties'].get('first',None)

    return data1

  dataset1 = ee.ImageCollection('MODIS/MYD09GA_006_NDVI').select('NDVI')
  dataset2 = ee.ImageCollection('MODIS/MYD09GA_006_NDWI').select('NDWI')
  dataset3 = ee.ImageCollection('MODIS/MOD09GA_006_EVI').select('EVI')
  dataset4 = ee.ImageCollection('MODIS/061/MOD11A1').select('LST_Day_1km')
  dataset5 = ee.ImageCollection('NASA/SMAP/SPL3SMP_E/005').select('tb_v_corrected_am')

  NDVI = dataload(dataset1)
  NDWI = dataload(dataset2)
  EVI = dataload(dataset3)
  LST = dataload(dataset4)*0.02
  Tb = dataload(dataset5)

  return NDVI,NDWI,EVI,LST,Tb

###########################################################加载############################################################################
###输入时间与经纬度信息、分辨率信息
year = int(input("请输入年份："))
month = int(input("请输入月份："))
day = int(input("请输入日期："))
lonn = float(input("请输入经度(°)："))
latt = float(input("请输入纬度(°)："))
scaled1 = int(input("请输入分辨率(m)："))
#year = 2018
#month = 6
#day = 17
#lonn = -100
#latt = 35
#scale = 500
Bare,Grass,Crops,Shrub,Tree,Clay,Sand,Bulk,Slope = auxiliarydate_load(lonn,latt,scaled1)
NDVI,NDWI,EVI,LST,Tb = modisload(lonn,latt,year,month,day,scaled1)

##############加载选择训练模型
m = int(input("请选择用于反演的模型：1.全球反演模型;2.分区域反演模型;3.分地表覆盖类型反演模型"))
if m == 1:
  model_w.load_state_dict(torch.load(str('/content/drive/My Drive/model_w_global.pth'),weights_only=True))
  model_tau.load_state_dict(torch.load(str('/content/drive/My Drive/model_tau_global.pth'),weights_only=True))
  model_SM.load_state_dict(torch.load(str('/content/drive/My Drive/model_SM_global.pth'),weights_only=True))
  model_w.eval();model_tau.eval();model_SM.eval()
if m == 2:
  if (latt >= 25 and latt <= 50) and (lonn >= -130 and lonn <= -70):
      model_w.load_state_dict(torch.load(str('/content/drive/My Drive/Global/model_w_CONUS.pth'),weights_only=True))
      model_tau.load_state_dict(torch.load(str('/content/drive/My Drive/Global/model_tau_CONUS.pth'),weights_only=True))
      model_SM.load_state_dict(torch.load(str('/content/drive/My Drive/Global/model_SM_CONUS.pth'),weights_only=True))
      model_w.eval();model_tau.eval();model_SM.eval()
  else:
      model_w.load_state_dict(torch.load(str('/content/drive/My Drive/Global/model_w_Other.pth'),weights_only=True))
      model_tau.load_state_dict(torch.load(str('/content/drive/My Drive/Global/model_tau_Other.pth'),weights_only=True))
      model_SM.load_state_dict(torch.load(str('/content/drive/My Drive/Global/model_SM_Other.pth'),weights_only=True))
      model_w.eval();model_tau.eval();model_SM.eval()
if m ==3:
  if Grass >= 0.5:
      model_w.load_state_dict(torch.load(str('/content/drive/My Drive/Global/model_w_Grass.pth'),weights_only=True))
      model_tau.load_state_dict(torch.load(str('/content/drive/My Drive/Global/model_tau_Grass.pth'),weights_only=True))
      model_SM.load_state_dict(torch.load(str('/content/drive/My Drive/Global/model_SM_Grass.pth'),weights_only=True))
      model_w.eval();model_tau.eval();model_SM.eval()
  elif Crops >= 0.5:
      model_w.load_state_dict(torch.load(str('/content/drive/My Drive/Global/model_w_Crops.pth'),weights_only=True))
      model_tau.load_state_dict(torch.load(str('/content/drive/My Drive/Global/model_tau_Crops.pth'),weights_only=True))
      model_SM.load_state_dict(torch.load(str('/content/drive/My Drive/Global/model_SM_Crops.pth'),weights_only=True))
      model_w.eval();model_tau.eval();model_SM.eval()
  elif Tree >= 0.5:
      model_w.load_state_dict(torch.load(str('/content/drive/My Drive/Global/model_w_Tree.pth'),weights_only=True))
      model_tau.load_state_dict(torch.load(str('/content/drive/My Drive/Global/model_tau_Tree.pth'),weights_only=True))
      model_SM.load_state_dict(torch.load(str('/content/drive/My Drive/Global/model_SM_Tree.pth'),weights_only=True))
      model_w.eval();model_tau.eval();model_SM.eval()
  else:
      model_w.load_state_dict(torch.load(str('/content/drive/My Drive/model_w_global.pth'),weights_only=True))
      model_tau.load_state_dict(torch.load(str('/content/drive/My Drive/model_tau_global.pth'),weights_only=True))
      model_SM.load_state_dict(torch.load(str('/content/drive/My Drive/model_SM_global.pth'),weights_only=True))
      model_w.eval();model_tau.eval();model_SM.eval()
####################################################反演####################################################################################
start = time()

if np.isnan(Tb):
  Tb = 300
if np.isnan(LST):
  LST = 300
Tb1 = torch.Tensor([Tb])
LST1 = torch.Tensor([LST])
sta = torch.cat([torch.Tensor([Grass]), torch.Tensor([Bare]), torch.Tensor([Crops]), torch.Tensor([Shrub]), torch.Tensor([Tree])], dim=0).reshape(1,-1)
dyn = torch.cat([torch.Tensor([NDVI]), torch.Tensor([NDWI]), torch.Tensor([EVI]), torch.Tensor([Grass]), torch.Tensor([Bare]), torch.Tensor([Crops]), torch.Tensor([Shrub]), torch.Tensor([Tree])], dim=0).reshape(1,-1)
dic = torch.cat([torch.Tensor([Grass]), torch.Tensor([Bare]), torch.Tensor([Crops]), torch.Tensor([Shrub]), torch.Tensor([Tree]), torch.Tensor([Clay]), torch.Tensor([Sand]), torch.Tensor([Bulk]),  torch.Tensor([Slope])], dim=0).reshape(1,-1)
print(dyn)
test_data=[NDVI,NDWI,EVI,LST,Tb,Bare,Grass,Crops,Shrub,Tree,Clay,Sand,Bulk,Slope]
if np.isnan(test_data).any():
  SM=np.nan
else:
  tau = model_tau(dyn)
  w = model_w(sta)
  input_test = SCA(w, tau, Tb1, LST1, dic)
  SM_test = model_SM(input_test)
  SM = SM_test.detach().numpy()[0,0]
print(SM)

end = time()

print("time: " + str(end - start) + " sec")
torch.cuda.empty_cache()

Mounted at /content/drive
请输入年份：2018
请输入月份：6
请输入日期：17
请输入经度(°)：-100
请输入纬度(°)：35
请输入分辨率(m)：500
请选择用于反演的模型：1.全球反演模型;2.分区域反演模型;3.分地表覆盖类型反演模型1
tensor([[ 0.1745, -0.0057,  0.3135,  0.5200,  0.0400,  0.1300,  0.1600,  0.1600]])
0.037006922
time: 0.11550116539001465 sec


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as tf
from torch.utils import data
import pandas as pd
import numpy as np
import datetime
from datetime import date
import time
import ee
ee.Authenticate()
ee.Initialize(project='ee-whulzh98')
from time import time
from PIL import Image

from google.colab import drive
drive.mount('/content/drive')

###########################################################读取模型##############################################################################
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.bn = nn.BatchNorm1d(input_size)
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size,hidden_size)
        self.fc3 = nn.Linear(hidden_size,hidden_size)
        self.fc4 = nn.Linear(hidden_size,hidden_size)
        self.fc5 = nn.Linear(hidden_size,output_size)

    def forward(self,x):
        x = self.bn(x)
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        x = torch.tanh(self.fc4(x))
        x = self.fc5(x)
        return x

#SCA-V模型
def SCA(w, tau, Tb, LST, dic):
    gamma = torch.exp(-tau / 0.766)    #cos40°=0.766
    sigma_veg = Tb - LST * ((1-w)*(1-gamma)+gamma)
    sigma_soil = LST * gamma * ((1-w)*(1-gamma)-1)
    r = sigma_veg / sigma_soil
    input = torch.cat([r, dic], dim=1)
    return input

model_w = Net(3,5,1)   #8 10 1
model_tau = Net(3,5,1)   #8 10 1
model_SM = Net(4,5,1)   #4 5 1

#模型加载
model_w.load_state_dict(torch.load(str('/content/drive/My Drive/model_w_global4.pth'),weights_only=True))
model_tau.load_state_dict(torch.load(str('/content/drive/My Drive/model_tau_global4.pth'),weights_only=True))
model_SM.load_state_dict(torch.load(str('/content/drive/My Drive/model_SM_global4.pth'),weights_only=True))
model_w.eval();model_tau.eval();model_SM.eval()
###########################################################数据获取##############################################################################

def auxiliarydate_load(roi):
  #加载 信息图像
  dataset1 = ee.Image("OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02").select('b0').unmask(0)
  dataset1 = dataset1.reproject(crs='EPSG:4326',scale=scaled).clip(roi)
  print(dataset1.getInfo())
  ft1 = dataset1.reduceRegion(ee.Reducer.toList(),roi,scaled,maxPixels=1e13)   #1000m分辨率
  Clay = np.array(ee.Array(ft1.get('b0')).getInfo()).reshape(-1,1)/100

  dataset2 = ee.Image("OpenLandMap/SOL/SOL_SAND-WFRACTION_USDA-3A1A1A_M/v02").select('b0').unmask(0)
  dataset2 = dataset2.reproject(crs='EPSG:4326',scale=scaled).clip(roi)
  ft2 = dataset2.reduceRegion(ee.Reducer.toList(),roi,scaled,maxPixels=1e13)   #1000m分辨率
  Sand = np.array(ee.Array(ft2.get('b0')).getInfo()).reshape(-1,1)/100

  dataset3 = ee.Image("OpenLandMap/SOL/SOL_BULKDENS-FINEEARTH_USDA-4A1H_M/v02").select('b0').unmask(0)
  dataset3 = dataset3.reproject(crs='EPSG:4326',scale=scaled).clip(roi)
  ft3 = dataset3.reduceRegion(ee.Reducer.toList(),roi,scaled,maxPixels=1e13)   #1000m分辨率
  Bulk = np.array(ee.Array(ft3.get('b0')).getInfo()).reshape(-1,1)/100

  dataset4 = ee.Image('USGS/SRTMGL1_003').select('elevation').unmask(0)
  dataset4 = dataset4.reproject(crs='EPSG:4326',scale=scaled).clip(roi)
  dataset4 = dataset4.addBands(ee.Image.pixelLonLat())
  ft4 = dataset4.reduceRegion(ee.Reducer.toList(),roi,scaled,maxPixels=1e13)   #1000m分辨率
  lat = np.array(ee.Array(ft4.get('latitude')).getInfo())
  lon = np.array(ee.Array(ft4.get('longitude')).getInfo())

  return lat,lon,Clay,Sand,Bulk

def modisload(roi,year,month,day):

  begin = datetime.date(year,month,day) #开始时间
  d = begin
  delta = datetime.timedelta(days=30)
  oneDate=d.strftime("%Y-%m-%d")
  dd=d+delta
  oneDate_next=dd.strftime("%Y-%m-%d")

  dataset1 = ee.ImageCollection('MODIS/MYD09GA_006_NDVI').filterBounds(roi).filterDate(oneDate,oneDate_next).select('NDVI').mean().unmask(0)
  dataset1 = dataset1.reproject(crs='EPSG:4326',scale=scaled).clip(roi)
  ft1 = dataset1.reduceRegion(ee.Reducer.toList(),roi,scaled,maxPixels=1e13)  #1000m分辨率
  NDVI = np.array(ee.Array(ft1.get('NDVI')).getInfo()).reshape(-1,1)

  dataset2 = ee.ImageCollection('MODIS/MYD09GA_006_NDWI').filterBounds(roi).filterDate(oneDate,oneDate_next).select('NDWI').mean().unmask(0)
  dataset2 = dataset2.reproject(crs='EPSG:4326',scale=scaled).clip(roi)
  ft2 = dataset2.reduceRegion(ee.Reducer.toList(),roi,scaled,maxPixels=1e13)  #1000m分辨率
  NDWI = np.array(ee.Array(ft2.get('NDWI')).getInfo()).reshape(-1,1)

  dataset3 = ee.ImageCollection('MODIS/061/MOD11A1').filterBounds(roi).filterDate(oneDate,oneDate_next).select('LST_Day_1km').mean().unmask(0)
  dataset3 = dataset3.reproject(crs='EPSG:4326',scale=scaled).clip(roi)
  ft3 = dataset3.reduceRegion(ee.Reducer.toList(),roi,scaled,maxPixels=1e13)  #1000m分辨率
  LST = np.array(ee.Array(ft3.get('LST_Day_1km')).getInfo()).reshape(-1,1)*0.02

  dataset4 = ee.ImageCollection('NASA/SMAP/SPL3SMP_E/005').filterBounds(roi).filterDate(oneDate,oneDate_next).select('tb_v_corrected_am').mean().unmask(0)
  dataset4 = dataset4.reproject(crs='EPSG:4326',scale=scaled).clip(roi)
  ft4 = dataset4.reduceRegion(ee.Reducer.toList(),roi,scaled)  #500m分辨率
  Tb = np.array(ee.Array(ft4.get('tb_v_corrected_am')).getInfo()).reshape(-1,1)

  dataset5 = ee.ImageCollection('MODIS/MOD09GA_006_EVI').filterBounds(roi).filterDate(oneDate,oneDate_next).select('EVI').mean().unmask(0)
  dataset5 = dataset5.reproject(crs='EPSG:4326',scale=scaled).clip(roi)
  ft5 = dataset5.reduceRegion(ee.Reducer.toList(),roi,scaled,maxPixels=1e13)  #1000m分辨率
  EVI = np.array(ee.Array(ft5.get('EVI')).getInfo()).reshape(-1,1)

  return NDVI,NDWI,LST,EVI,Tb

##########################################################图像转换#######################################################################
def toArray(lat,lon,SM):
  uniqueLats,lat_ind = np.unique(lat,return_inverse=True)
  uniqueLons,lon_ind = np.unique(lon,return_inverse=True)
  ncols = len(uniqueLons)
  nrows = len(uniqueLats)
  print(ncols,nrows)
  arr = np.zeros([nrows,ncols])

  for x in range(0,len(SM)):
    arr[nrows-1-lat_ind[x],lon_ind[x]]=SM[x]
  return arr

###########################################################反演############################################################################
#中国经纬度范围(73.55, 18.16, 135.08, 53.55)
#世界经纬度范围(-180,-57,180,84)
roi = ee.Geometry.Rectangle(73.55, 18.16, 135.08, 53.55).bounds()  #注意经纬度差不能超过90°
#roi = ee.Geometry.Polygon([[[0, 84],[0, -57],[179, -57],[179, 84]]]).bounds() #注意坐标应按照顺时针，经度不能为180
#roi = ee.Geometry.Rectangle(-45, -45,45, 45).bounds()   #注意经纬度差不能超过90°

year = 2018
month = 7
day = 1

scaled = 9000

start = time()

lat,lon,Clay,Sand,Bulk = auxiliarydate_load(roi)
NDVI,NDWI,LST,EVI,Tb = modisload(roi,year,month,day)
Tb[np.isnan(Tb)]=300
LST[np.isnan(LST)]=300
NDVI[np.isnan(NDVI)]=0
NDWI[np.isnan(NDWI)]=0
EVI[np.isnan(EVI)]=0
Clay[np.isnan(Clay)]=0
Sand[np.isnan(Sand)]=0
Bulk[np.isnan(Bulk)]=0

end = time()

print("time: " + str(end - start) + " sec")

start = time()

Tb1 = torch.Tensor(Tb)
LST1 = torch.Tensor(LST)
sta = torch.cat([torch.Tensor(NDVI), torch.Tensor(NDWI), torch.Tensor(EVI)], dim=1)
dyn = torch.cat([torch.Tensor(NDVI), torch.Tensor(NDWI), torch.Tensor(EVI)], dim=1)
dic = torch.cat([torch.Tensor(Clay), torch.Tensor(Sand), torch.Tensor(Bulk)], dim=1)

del Tb,LST,NDWI,EVI,Clay,Bulk

tau = model_tau(dyn)
w = model_w(sta)
del sta,dyn
del model_w,model_tau
input_test = SCA(w, tau, Tb1, LST1, dic)
del w,tau,Tb1,LST1,dic
SM_test = model_SM(input_test)
del input_test
del model_SM
torch.cuda.empty_cache()
SM = SM_test.detach().numpy()

end = time()

print("time: " + str(end - start) + " sec")

start = time()

SM[Sand == 0]=0
#SM[NDVI <= 0]=0

SM1 = toArray(lat,lon,SM)
SM_img=Image.fromarray(SM1)
SM_img.save('/content/drive/My Drive/Global.tif')

end = time()

print("time: " + str(end - start) + " sec")

Mounted at /content/drive
{'type': 'Image', 'bands': [{'id': 'b0', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [762, 489], 'origin': [909, -713], 'crs': 'EPSG:4326', 'crs_transform': [0.08084837557075693, 0, 0, 0, -0.08084837557075693, 0]}], 'version': 1651671975432479, 'id': 'OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02', 'properties': {'system:footprint': {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[73.55, 18.159999999999975], [135.08, 18.159999999999975], [135.08, 57.59798889543937], [73.55, 57.59798889543937], [73.55, 18.159999999999975]]]}, 'system:time_start': -631152000000, 'system:description': 'Based on machine learning predictions from global compilation of soil profiles and samples.\n<br>\n<a href="https://github.com/Envirometrix/LandGISmaps#soil-properties-and-classes"> description </a>', 'system:time_end': 1514678400000, 'system:asset_size': 10811502500}}
time: 70.83998966217041 sec
time: 0.434173822402954

<ipython-input-2-2c4c8296d0bb>:132: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  arr[nrows-1-lat_ind[x],lon_ind[x]]=SM[x]


time: 0.7895209789276123 sec


In [ ]:
###反演图生产###
from google.colab import drive
drive.mount('/content/drive')

import torch
import torch.nn as nn
import torch.nn.functional as tf
from torch.utils import data
from osgeo import gdal
import pandas as pd
import os
import numpy as np
import random
from PIL import Image

class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.bn = nn.BatchNorm1d(input_size)
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size,hidden_size)
        self.fc3 = nn.Linear(hidden_size,hidden_size)
        self.fc4 = nn.Linear(hidden_size,hidden_size)
        self.fc5 = nn.Linear(hidden_size,output_size)

    def forward(self,x):
        x = self.bn(x)
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        x = torch.tanh(self.fc4(x))
        x = self.fc5(x)
        return x

def SCA(w, tau, Tb, LST, dic):
    gamma = torch.exp(-tau / 0.766)    #cos40°=0.766
    sigma_veg = Tb - LST * ((1-w)*(1-gamma)+gamma)
    sigma_soil = LST * gamma * ((1-w)*(1-gamma)-1)
    r = sigma_veg / sigma_soil
    input = torch.cat([r, dic], dim=1)
    return input


if __name__ == '__main__':
  dataset1=gdal.Open(r'/content/drive/My Drive/Global/Tb10Img9km.tif')
  dataset2=gdal.Open(r'/content/drive/My Drive/Global/LST10Img9km.tif')
  dataset3=gdal.Open(r'/content/drive/My Drive/Global/NDVI10Img9km.tif')
  dataset4=gdal.Open(r'/content/drive/My Drive/Global/NDWI10Img9km.tif')
  dataset5=gdal.Open(r'/content/drive/My Drive/Global/EVI10Img9km.tif')
  dataset6=gdal.Open(r'/content/drive/My Drive/Global/grassImg9km.tif')
  dataset7=gdal.Open(r'/content/drive/My Drive/Global/bareImg9km.tif')
  dataset8=gdal.Open(r'/content/drive/My Drive/Global/cropsImg9km.tif')
  dataset9=gdal.Open(r'/content/drive/My Drive/Global/shrubImg9km.tif')
  dataset10=gdal.Open(r'/content/drive/My Drive/Global/treeImg9km.tif')
  dataset11=gdal.Open(r'/content/drive/My Drive/Global/clayImg9km.tif')
  dataset12=gdal.Open(r'/content/drive/My Drive/Global/sandImg9km.tif')
  dataset13=gdal.Open(r'/content/drive/My Drive/Global/bulkImg9km.tif')

  im_width=dataset12.RasterXSize
  im_height=dataset12.RasterYSize
  print(im_width)
  print(im_height)
  im_geotrans=dataset12.GetGeoTransform()
  im_proj=dataset12.GetProjection()

  Tb=dataset1.ReadAsArray(0,0,im_width,im_height).reshape(-1,1)
  LST=dataset2.ReadAsArray(0,0,im_width,im_height).reshape(-1,1)*0.02
  NDVI=dataset3.ReadAsArray(0,0,im_width,im_height).reshape(-1,1)
  NDWI=dataset4.ReadAsArray(0,0,im_width,im_height).reshape(-1,1)
  EVI=dataset5.ReadAsArray(0,0,im_width,im_height).reshape(-1,1)
  Grass=(dataset6.ReadAsArray(0,0,im_width,im_height)/100).reshape(-1,1)
  Bare=(dataset7.ReadAsArray(0,0,im_width,im_height)/100).reshape(-1,1)
  Crops=(dataset8.ReadAsArray(0,0,im_width,im_height)/100).reshape(-1,1)
  Shrub=(dataset9.ReadAsArray(0,0,im_width,im_height)/100).reshape(-1,1)
  Tree=(dataset10.ReadAsArray(0,0,im_width,im_height)/100).reshape(-1,1)
  Clay=(dataset11.ReadAsArray(0,0,im_width,im_height)/100).reshape(-1,1)
  Sand=(dataset12.ReadAsArray(0,0,im_width,im_height)/100).reshape(-1,1)
  Bulk=(dataset13.ReadAsArray(0,0,im_width,im_height)/100).reshape(-1,1)

  del dataset1,dataset2,dataset3,dataset4,dataset5,dataset6,dataset7
  del dataset8,dataset9,dataset10,dataset11,dataset12,dataset13

  Tb[np.isnan(Tb)]=300
  LST[np.isnan(LST)]=300
  NDVI[np.isnan(NDVI)]=0
  NDWI[np.isnan(NDWI)]=0
  EVI[np.isnan(EVI)]=0
  Grass[np.isnan(Grass)]=0
  Bare[np.isnan(Bare)]=0
  Crops[np.isnan(Crops)]=0
  Shrub[np.isnan(Shrub)]=0
  Tree[np.isnan(Tree)]=0
  Clay[np.isnan(Clay)]=0
  Sand[np.isnan(Sand)]=0
  Bulk[np.isnan(Bulk)]=0

###########################################################################################################################################

  model_w = Net(8,10,1)   #5 10 1
  model_tau = Net(8,10,1)   #8 10 1
  model_SM = Net(4,5,1)   #10 10 1

  #模型加载
  model_w.load_state_dict(torch.load(str('/content/drive/My Drive/model_w_global3.pth'),weights_only=True))
  model_tau.load_state_dict(torch.load(str('/content/drive/My Drive/model_tau_global3.pth'),weights_only=True))
  model_SM.load_state_dict(torch.load(str('/content/drive/My Drive/model_SM_global3.pth'),weights_only=True))
  model_w.eval();model_tau.eval();model_SM.eval()

  Tb1 = torch.Tensor(Tb)
  LST1 = torch.Tensor(LST)
  sta = torch.cat([torch.Tensor(NDVI), torch.Tensor(NDWI), torch.Tensor(EVI), torch.Tensor(Grass), torch.Tensor(Bare), torch.Tensor(Crops), torch.Tensor(Shrub), torch.Tensor(Tree)], dim=1)
  dyn = torch.cat([torch.Tensor(NDVI), torch.Tensor(NDWI), torch.Tensor(EVI), torch.Tensor(Grass), torch.Tensor(Bare), torch.Tensor(Crops), torch.Tensor(Shrub), torch.Tensor(Tree)], dim=1)
  dic = torch.cat([torch.Tensor(Clay), torch.Tensor(Sand), torch.Tensor(Bulk)], dim=1)

  del Tb,LST,NDWI,EVI,Grass,Bare,Crops,Shrub,Tree,Clay,Bulk

  tau = model_tau(dyn)
  w = model_w(sta)
  del sta,dyn
  del model_w,model_tau
  torch.cuda.empty_cache()
  input_test = SCA(w, tau, Tb1, LST1, dic)
  del w,tau,Tb1,LST1,dic
  SM_test = model_SM(input_test)
  del input_test
  del model_SM
  torch.cuda.empty_cache()
  SM = SM_test.detach().numpy()


  SM = np.array(SM)*6/5
  for h in range(0,im_width*im_height):
    if NDVI[h] <= 0 or Sand[h] == 0:
      SM[h,0]=0
  SM = SM.reshape(im_height,im_width)
  SM = np.array(SM, dtype=np.float32)


  ##Array转Tif

  driver=gdal.GetDriverByName('GTiff')
  datasetnew=driver.Create('/content/drive/My Drive/GlobalSMImg10_3.tif',im_width,im_height,1,gdal.GDT_Float32)
  datasetnew.SetGeoTransform(im_geotrans)
  datasetnew.SetProjection(im_proj)
  band=datasetnew.GetRasterBand(1)
  band.WriteArray(SM)
  datasetnew.FlushCache()
  del datasetnew

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
4453
1745
